In [68]:
!pip install tensorflow-hub

     |████████████████████████████████| 107 kB 6.5 MB/s eta 0:00:01


In [69]:
import tensorflow_hub as hub

In [1]:
import pandas as pd

pd.set_option('max_columns', 500)

df = pd.read_csv('/home/aschharwood/notebooks/covid/feature_target_v1_county_nyt_cdc_moa_tweets_gdelt_apple.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)

df.info(max_cols=500)

mean = df._get_numeric_data().mean()

df.fillna(mean, inplace=True)

df.head()

df['COUNTYFP'] = df['COUNTYFP'].astype('string')

df.describe()

import numpy as np

df['always_binary'] = np.where(df['ALWAYS']>.50, 1, 0)

df['always_binary'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 174 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COUNTYFP         3142 non-null   int64  
 1   NEVER            3142 non-null   float64
 2   RARELY           3142 non-null   float64
 3   SOMETIMES        3142 non-null   float64
 4   FREQUENTLY       3142 non-null   float64
 5   ALWAYS           3142 non-null   float64
 6   text_tokens_str  3142 non-null   object 
 7   ST_ABBR          3142 non-null   object 
 8   FIPS             3142 non-null   int64  
 9   AREA_SQMI        3142 non-null   float64
 10  E_TOTPOP         3142 non-null   int64  
 11  M_TOTPOP         3142 non-null   int64  
 12  E_HU             3142 non-null   int64  
 13  M_HU             3142 non-null   int64  
 14  E_HH             3142 non-null   int64  
 15  M_HH             3142 non-null   int64  
 16  E_POV            3142 non-null   int64  
 17  M_POV        

0    1598
1    1544
Name: always_binary, dtype: int64

In [3]:
df.fillna(df._get_numeric_data().mean(), inplace=True)

In [5]:
df.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 175 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COUNTYFP         3142 non-null   string 
 1   NEVER            3142 non-null   float64
 2   RARELY           3142 non-null   float64
 3   SOMETIMES        3142 non-null   float64
 4   FREQUENTLY       3142 non-null   float64
 5   ALWAYS           3142 non-null   float64
 6   text_tokens_str  3142 non-null   object 
 7   ST_ABBR          3142 non-null   object 
 8   FIPS             3142 non-null   int64  
 9   AREA_SQMI        3142 non-null   float64
 10  E_TOTPOP         3142 non-null   int64  
 11  M_TOTPOP         3142 non-null   int64  
 12  E_HU             3142 non-null   int64  
 13  M_HU             3142 non-null   int64  
 14  E_HH             3142 non-null   int64  
 15  M_HH             3142 non-null   int64  
 16  E_POV            3142 non-null   int64  
 17  M_POV        

In [6]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

# Tweets Model

In [10]:
df['text_tokens_str']

0       Sneak in , sip some suds , slam some suds ... ...
1                                                    None
2                                                    None
3       Listen , I don ’ t know if masks work or not ,...
4       Good morning Sexy ! Don ’ t forget to eat your...
                              ...                        
3137                                                 None
3138                                                 None
3139                                                 None
3140                                                 None
3141                                                 None
Name: text_tokens_str, Length: 3142, dtype: object

In [11]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [13]:
X = []
sentences = list(df['text_tokens_str'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [23]:
y = df['always_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
maxlen=200
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [54]:
#!wget https://nlp.stanford.edu/data/wordvecs/glove.twitter.27B.zip

In [31]:
# import zipfile
# with zipfile.ZipFile('/home/aschharwood/notebooks/covid/notebooks/glove.twitter.27B.zip', 'r') as zip_ref:
#     zip_ref.extractall('/home/aschharwood/notebooks/covid/notebooks/glove_tweets')

In [33]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/home/aschharwood/notebooks/covid/notebooks/glove_tweets/glove.twitter.27B.100d.txt')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [34]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [45]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=True)(deep_inputs)
LSTM_layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(1, activation='softmax')(LSTM_layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [41]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 100)          3303000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,420,377
Trainable params: 3,420,377
Non-trainable params: 0
_________________________________________________________________


In [46]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=[X_test, y_test])

Epoch 1/5
20/20 [==============================] - 4s 222ms/step - loss: 7.8825 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/5
20/20 [==============================] - 4s 209ms/step - loss: 7.8825 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/5
20/20 [==============================] - 4s 211ms/step - loss: 7.8825 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 4/5
20/20 [==============================] - 4s 210ms/step - loss: 7.8825 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 5/5
20/20 [==============================] - 4s 211ms/step - loss: 7.8825 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00


### Tweets Pretrained Google News

In [85]:
X = df['text_tokens_str'].apply(lambda x: preprocess_text(x))

y = df['always_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


#define feed forward neural network with one embed layer and 4 dense layers
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

#train model
print('training model')
history = model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

model defined and compiled
training model
Epoch 1/5
79/79 [==============================] - 54s 688ms/step - loss: 0.6590 - accuracy: 0.6013 - val_loss: 0.6365 - val_accuracy: 0.6010
Epoch 2/5
79/79 [==============================] - 54s 684ms/step - loss: 0.6233 - accuracy: 0.6486 - val_loss: 0.6283 - val_accuracy: 0.6105
Epoch 3/5
79/79 [==============================] - 54s 686ms/step - loss: 0.5908 - accuracy: 0.6761 - val_loss: 0.6308 - val_accuracy: 0.6153
Epoch 4/5
79/79 [==============================] - 54s 684ms/step - loss: 0.5622 - accuracy: 0.6948 - val_loss: 0.6443 - val_accuracy: 0.6137
Epoch 5/5
79/79 [==============================] - 54s 685ms/step - loss: 0.5497 - accuracy: 0.6980 - val_loss: 0.6778 - val_accuracy: 0.6216
model training complete


# News Data - High Sentiment

In [75]:
X = []
sentences = list(df['High'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [55]:
y = df['always_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
maxlen=200
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [56]:
embeddings_dictionary = dict()

glove_file = open('/home/aschharwood/notebooks/covid/notebooks/glove_tweets/glove.twitter.27B.100d.txt')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [57]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [61]:
from keras.losses import BinaryCrossentropy

In [62]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=True)(deep_inputs)
LSTM_layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(1, activation='softmax')(LSTM_layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss=BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['acc'])

In [63]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_data=[X_test, y_test])

Epoch 1/5
20/20 [==============================] - 4s 214ms/step - loss: 0.8302 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/5
20/20 [==============================] - 4s 198ms/step - loss: 0.8302 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 3/5
20/20 [==============================] - 4s 199ms/step - loss: 0.8302 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 4/5
20/20 [==============================] - 4s 200ms/step - loss: 0.8302 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 5/5
20/20 [==============================] - 4s 198ms/step - loss: 0.8302 - acc: 0.4831 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00


### News - High - Google News Pretrained Model

In [71]:
import tensorflow as tf

In [72]:
#download and set word vector pretrained model
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

print('Google news model downlownded')

Google news model downlownded


In [82]:
X = df['High'].apply(lambda x: preprocess_text(x))

y = df['always_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


#define feed forward neural network with one embed layer and 4 dense layers
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

#train model
print('training model')
history = model.fit(X_train, y_train, epochs=3, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

# New - Low

In [83]:
X = df['Low'].apply(lambda x: preprocess_text(x))

y = df['always_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


#define feed forward neural network with one embed layer and 4 dense layers
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

#train model
print('training model')
history = model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

model defined and compiled
training model
Epoch 1/5
79/79 [==============================] - 55s 694ms/step - loss: 0.6598 - accuracy: 0.5169 - val_loss: 0.6231 - val_accuracy: 0.4754
Epoch 2/5
79/79 [==============================] - 55s 691ms/step - loss: 0.6353 - accuracy: 0.5169 - val_loss: 0.6126 - val_accuracy: 0.4754
Epoch 3/5
79/79 [==============================] - 55s 691ms/step - loss: 0.6428 - accuracy: 0.5169 - val_loss: 0.6167 - val_accuracy: 0.4754
Epoch 4/5
79/79 [==============================] - 55s 692ms/step - loss: 0.6359 - accuracy: 0.5169 - val_loss: 0.6083 - val_accuracy: 0.4754
Epoch 5/5
79/79 [==============================] - 55s 693ms/step - loss: 0.6272 - accuracy: 0.5169 - val_loss: 0.5980 - val_accuracy: 0.4754
model training complete


# Meta Data

In [89]:
df.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 175 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   COUNTYFP         3142 non-null   string 
 1   NEVER            3142 non-null   float64
 2   RARELY           3142 non-null   float64
 3   SOMETIMES        3142 non-null   float64
 4   FREQUENTLY       3142 non-null   float64
 5   ALWAYS           3142 non-null   float64
 6   text_tokens_str  3142 non-null   object 
 7   ST_ABBR          3142 non-null   object 
 8   FIPS             3142 non-null   int64  
 9   AREA_SQMI        3142 non-null   float64
 10  E_TOTPOP         3142 non-null   int64  
 11  M_TOTPOP         3142 non-null   int64  
 12  E_HU             3142 non-null   int64  
 13  M_HU             3142 non-null   int64  
 14  E_HH             3142 non-null   int64  
 15  M_HH             3142 non-null   int64  
 16  E_POV            3142 non-null   int64  
 17  M_POV        

In [90]:
df_num_cat = df.drop(['NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 'text_tokens_str', 'FIPS', 'High', 'Low'], axis=1)

In [91]:
df_cat = df_num_cat[['COUNTYFP', 'ST_ABBR']]

In [124]:
df_num = df_num_cat.drop(['COUNTYFP', 'ST_ABBR', 'always_binary'], axis=1)

# Category Model

In [96]:
X = df_cat.values

array([['5001', 'AR'],
       ['5003', 'AR'],
       ['5005', 'AR'],
       ...,
       ['56041', 'WY'],
       ['56043', 'WY'],
       ['56045', 'WY']], dtype=object)

In [97]:
from sklearn import preprocessing

In [98]:
label_encoder = preprocessing.OneHotEncoder()

In [101]:
x_coded = label_encoder.fit_transform(df_cat)

In [102]:
x_coded

<3142x3193 sparse matrix of type '<class 'numpy.float64'>'
	with 6284 stored elements in Compressed Sparse Row format>

In [104]:
X_train, X_test, y_train, y_test = train_test_split(x_coded, y, test_size=0.20, random_state=42)


In [113]:
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

In [114]:
X_test = pd.DataFrame.sparse.from_spmatrix(X_test)

In [115]:
input2 = Input(shape=(3193,))
dense_layer_1 = Dense(10, activation='relu')(input2)
dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
output = Dense(1, activation='softmax')(dense_layer_2)

model = Model(inputs=input2, outputs=output)
#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

#train model
print('training model')
history = model.fit(X_train, y_train, epochs=5, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

model defined and compiled
training model
Epoch 1/5
79/79 [==============================] - 1s 13ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 2/5
79/79 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 3/5
79/79 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 4/5
79/79 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 5/5
79/79 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
model training complete


# Num Model

In [117]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [125]:
scaler = MinMaxScaler()
scaled_df_num = scaler.fit_transform(df_num)

In [132]:
scaler = StandardScaler()
scaled_df_num = scaler.fit_transform(df_num)

In [133]:
X_train, X_test, y_train, y_test = train_test_split(df_num, y, test_size=0.20, random_state=42)


In [134]:
X_train.shape

(2513, 163)

In [135]:
input2 = Input(shape=(163,))
dense_layer_1 = Dense(163, activation='relu')(input2)
dense_layer_2 = Dense(80, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(10, activation='relu')(dense_layer_2)
dense_layer_4 = Dense(10, activation='relu')(dense_layer_3)
output = Dense(1, activation='softmax')(dense_layer_4)

model = Model(inputs=input2, outputs=output)
#compile model
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
print('model defined and compiled')

#train model
print('training model')
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test, y_test))
print('model training complete')

model defined and compiled
training model
Epoch 1/100
79/79 [==============================] - 0s 3ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 2/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 3/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 4/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 5/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 6/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0.7886 - val_accuracy: 0.5246
Epoch 7/100
79/79 [==============================] - 0s 1ms/step - loss: 0.8302 - accuracy: 0.4831 - val_loss: 0